In [77]:
import sys
sys.path.append('/home/chris/repos/financial_analysis/')
import pandas as pd

from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
from credentials import *
api_key = key

In [176]:
indices = [
    '^GDAXI',#DAX
    '^GSPC', #S&P 500
    '^DJI',  #DowJones 30
    #'^IXIC', #NASDAQ
    '^NYA', #NYSE COMPOSITE (DJ)
    '^XAX',  #NYSE AMEX COMPOSITE INDEX
    #'^BATSK',#BATS 1000 Index
   # '^RUT',  #Russell 2000
   # '^FTSE', #FTSE 100
    '^FCHI', #CAC 40
    #'^STOXX50E', #STOXX 50
    '^N100', #EURONEXT 100
    '^BFX',  #BEL 20 Brussels
    '^N225', #Nikkei
    '^HSI',  #Hangseng   
]


assets_portfolio = [
    'SIE.DE',
    'EOAN.DE',
    'DTE.DE',
    'FTK.DE',
    'LHA.DE'
]


assets = indices + assets_portfolio


In [177]:
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
df_price = pd.DataFrame()
df_bbands = pd.DataFrame()
df_adx = pd.DataFrame()
df_cci = pd.DataFrame()
df_aroon = pd.DataFrame()


for asset in assets:
    ts = TimeSeries(key=api_key, output_format='pandas')
    data, meta_data = ts.get_daily(symbol=asset, outputsize='full')
    data = data.drop('5. volume',axis=1)
    data = data.drop(data.loc[(data==0).any(axis=1)].index)  
    df_price['close_'+asset.strip('^')] = data['4. close']
    data['4. close'].plot()
    plt.title('End of day Times Series for ' + asset.strip('^') )
    plt.show()
    #Read BBand technical indicator
    ti = TechIndicators(key=key, output_format='pandas')
    data, meta_data = ti.get_bbands(symbol=asset, interval='daily', time_period=200)
    df_bbands[data.columns[0] + ' ' + asset.strip('^')] = data[data.columns[0]]
    df_bbands[data.columns[1] + ' ' + asset.strip('^')] = data[data.columns[1]]
    df_bbands[data.columns[2] + ' ' + asset.strip('^')] = data[data.columns[2]]
    data.plot()
    plt.title('BBbands indicator for ' +  asset.strip('^') + ' stock (daily)')
    plt.show()
    #Read ADX technical indicator
    #ti = TechIndicators(key=key, output_format='pandas')
    data, meta_data = ti.get_adx(symbol=asset, interval='daily', time_period=20)
    df_adx[data.columns[0] + ' ' + asset.strip('^')] = data[data.columns[0]]
    #Read CCI technical indicator
    #ti = TechIndicators(key=key, output_format='pandas')
    data, meta_data = ti.get_cci(symbol=asset, interval='daily', time_period=20)
    df_cci[data.columns[0] + ' ' + asset.strip('^')] = data[data.columns[0]]
    #Read AROON technical indicator
    #ti = TechIndicators(key=key, output_format='pandas')
    data, meta_data = ti.get_aroon(symbol=asset, interval='daily', time_period=20)
    df_aroon[data.columns[0] + ' ' + asset.strip('^')] = data[data.columns[0]]

ChunkedEncodingError: ('Connection broken: IncompleteRead(5561 bytes read, 4679 more expected)', IncompleteRead(5561 bytes read, 4679 more expected))

### Concat the dataframes for the last 100 days of trading

In [ ]:
df_aroon  = df_aroon.iloc[-100:]
df_cci    = df_cci.iloc[-100:]
df_adx    = df_adx.iloc[-100:]
df_bbands = df_bbands.iloc[-100:]
df_price  = df_price.iloc[-100:]
df_all = pd.concat([df_price,df_aroon, df_cci,df_adx,df_bbands], axis=1)

In [ ]:
df_all = df_all.fillna(method='bfill')

## Check for correlations 
* Check for correlations between particular end of day prices
* Check for correlations among the technical indicators 

### The Average Directional Movement Index (ADX) indicator
* Compares the high and low of subsequent days of trade 
* I.e. if the highest price of a day exceeds the high of the previous day, an upward trend is associated 

In [ ]:
plt.figure(figsize=(10,10))
plt.title('Correlation: ADX', y=1.05, size=15)
plot = sns.heatmap(df_adx.corr(), xticklabels=df_adx.columns, yticklabels=df_adx.columns,  linewidths=0.1,vmax=1.0,vmin=0, square=True, linecolor='white')
#plot.set_xticklabels(plot.get_xticklabels(),rotation=30)
#plot.set_ylabel('')
#plot.set_xlabel('')
plt.setp(plot.get_xticklabels(), rotation=45)
plt.setp(plot.get_yticklabels(), rotation=0)
plt.show()

### Commodity Channel Index 
* The CCI is designed to detect beginning and ending market trends.
* If the price is making new highs, and the CCI is not, then a price correction is likely.

In [ ]:
plt.figure(figsize=(10,10))
plt.title('Correlation: CCI', y=1.05, size=15)
plot = sns.heatmap(df_cci.corr(), xticklabels=df_cci.columns, yticklabels=df_cci.columns,  linewidths=0.1,vmax=1.0,vmin=0, square=True, linecolor='white')
#plot.set_xticklabels(plot.get_xticklabels(),rotation=30)
#plot.set_ylabel('')
#plot.set_xlabel('')
plt.setp(plot.get_xticklabels(), rotation=45)
plt.setp(plot.get_yticklabels(), rotation=0)
plt.show()

### Aroon indicator
* The Aroon indicator attempts to show when a new trend is dawning.
* When the Aroon Up is staying between 70 and 100 then it indicates an upward trend. When the Aroon Down is staying between 70 and 100 then it indicates an downward trend. A strong upward trend is indicated when the Aroon Up is above 70 while the Aroon Down is below 30. Likewise, a strong downward trend is indicated when the Aroon Down is above 70 while the Aroon Up is below 30. Also look for crossovers. 

In [ ]:
plt.figure(figsize=(10,10))
plt.title('Correlation: Aroon', y=1.05, size=15)
plot = sns.heatmap(df_aroon.corr(), xticklabels=df_aroon.columns, yticklabels=df_aroon.columns,  linewidths=0.1,vmax=1.0,vmin=0, square=True, linecolor='white')
#plot.set_xticklabels(plot.get_xticklabels(),rotation=30)
#plot.set_ylabel('')
#plot.set_xlabel('')
plt.setp(plot.get_xticklabels(), rotation=45)
plt.setp(plot.get_yticklabels(), rotation=0)
plt.show()

### Predicitive model based on LSTM

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
from math import sqrt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import LSTM,Dense

from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
np.random.seed(7)

In [ ]:
df_all.columns[14]

In [ ]:
#Define asset to be predicted
predicted_asset = 'close_EOAN.DE'
asset_position = list(df_all.columns).index(predicted_asset)
print(asset_position)

In [ ]:
cols = list(df_all)
cols.insert(0, cols.pop(cols.index(predicted_asset)))
df_all = df_all.loc[:, cols]

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
 # load dataset
dataset = df_all
values = dataset.values
# integer encode direction
encoder = LabelEncoder()
values[:,4] = encoder.fit_transform(values[:,4])
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)
# drop columns we don't want to predict
reframed.drop(reframed.iloc[:, df_all.shape[1]+1:reframed.shape[1]], axis=1, inplace=True)
print(reframed.head())

In [ ]:
# split into train and test sets
values = reframed.values
train = values[:40, :]
test = values[40:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

In [ ]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
#model.add(Dense(1, activation='tanh'))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=100, batch_size=10, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
yhat = model.predict(test_X)
#test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X.reshape((test_X.shape[0], test_X.shape[2]))[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)

In [ ]:
yhat = model.predict(test_X)

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
historic = np.array(df_all[predicted_asset])
future = inv_yhat
prediction = np.append(historic,future)  

In [ ]:
plt.axvline(x=len(historic), linestyle='--', color='red')
plt.plot(prediction)

In [ ]:
future[0],historic.mean()

In [ ]:
future.min(), future.max()

In [ ]:
'''def predictive_model(test_X)
    for neuron_number in np.linspace(20,1000,50)
        # design network
        model = Sequential()
        model.add(LSTM(neuron_number, input_shape=(train_X.shape[1], train_X.shape[2])))
        model.add(Dense(1))
        #model.add(Dense(1, activation='tanh'))
        model.compile(loss='mae', optimizer='adam')
        # fit network
        history = model.fit(train_X, train_y, epochs=200, batch_size=40, validation_data=(test_X, test_y), verbose=2, shuffle=True)
        # plot history
        pyplot.plot(history.history['loss'], label='train')
        pyplot.plot(history.history['val_loss'], label='test')
        pyplot.legend()
        pyplot.show()
        yhat = model.predict(test_X)'''